<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/SMA_Short_VIX_Adjust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SMA_Short_VIX_Adjust
---
This notebook analyzes historical price data (specifically for UVXY) to evaluate a short-volatility strategy triggered by **Simple Moving Average (SMA)** crossovers, filtered by a volatility threshold.

* **Data Acquisition & Integration**: It downloads historical data for the target ticker and the VIX Index (`^VIX`) using `yfinance`, aligning both datasets to ensure consistent trading days.
* **SMA Signal Logic**: It identifies "sign changes" where the closing price crosses above or below the SMA. A bearish entry signal is generated when the price drops below the SMA (sign change to -1).
* **Volatility Filtering**: The strategy only executes "Short" trades if the VIX is above a user-defined limit (e.g., 17), targeting periods of heightened volatility where mean reversion is more likely.
* **Trade Performance Tracking**: For every valid entry, the script tracks the return until the next bullish SMA crossover (exit). It calculates the total return, trade count, success count (win rate), and yearly performance.
* **Sensitivity Analysis**: The script iterates through a range of SMA lookback periods (from 3 to 21) to compare how different moving average lengths impact total returns and win rates over time.
* **Comparative Summary**: The final output includes pivoted tables for yearly returns, trade counts, and win rates, alongside statistical summaries including Grand Totals, Standard Deviation, and Mean Absolute Deviation (MAD) for risk assessment.

---

In [56]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define a ticker and a date range for your data
ticker = 'UVXY'
start_date = '2012-01-01'
end_date = '2025-12-25'
window = 5
lookback_range = [3,21]
vix_limit = 17

# Download historical data from Yahoo Finance for a single ticker.
# This will result in a DataFrame with 'Date' as a simple index.
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

# Use reset_index() to convert the 'Date' index into a column.
data = data.reset_index()

# Now, to get a new DataFrame with just the 'Price' level, we can use droplevel()
# This removes the 'Ticker' level from the columns, leaving only the 'Price' level.
data = data.droplevel(level='Ticker', axis=1)

# Now, add the 'Ticker' column at position 1 (right after the 'Date' column).
data.insert(1, 'Ticker', ticker)

# The DataFrame is now a flat table with no MultiIndex.
display(data)

[*********************100%***********************]  1 of 1 completed


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2012-01-03,UVXY,1.632000e+11,1.675500e+11,1.612500e+11,1.656000e+11,0
1,2012-01-04,UVXY,1.569000e+11,1.705500e+11,1.564500e+11,1.678500e+11,0
2,2012-01-05,UVXY,1.497000e+11,1.642500e+11,1.492500e+11,1.611000e+11,0
3,2012-01-06,UVXY,1.449000e+11,1.530000e+11,1.419000e+11,1.471500e+11,0
4,2012-01-09,UVXY,1.405500e+11,1.446000e+11,1.390500e+11,1.422000e+11,0
...,...,...,...,...,...,...,...
3511,2025-12-18,UVXY,4.108000e+01,4.255000e+01,4.060000e+01,4.165000e+01,10864500
3512,2025-12-19,UVXY,3.866000e+01,4.045000e+01,3.853000e+01,4.040000e+01,6814300
3513,2025-12-22,UVXY,3.680000e+01,3.784000e+01,3.667000e+01,3.754000e+01,5383700
3514,2025-12-23,UVXY,3.698000e+01,3.716000e+01,3.661000e+01,3.708000e+01,4483100


In [57]:
# Download historical data from Yahoo Finance for a single ticker.
# This will result in a DataFrame with 'Date' as a simple index.
vix_ticker = '^VIX'
vix_data = yf.download(vix_ticker, start=start_date, end=end_date, auto_adjust=True)

# Use reset_index() to convert the 'Date' index into a column.
vix_data = vix_data.reset_index()

# Now, to get a new DataFrame with just the 'Price' level, we can use droplevel()
# This removes the 'Ticker' level from the columns, leaving only the 'Price' level.
vix_data = vix_data.droplevel(level='Ticker', axis=1)

# Now, add the 'Ticker' column at position 1 (right after the 'Date' column).
vix_data.insert(1, 'Ticker', vix_ticker)

# The DataFrame is now a flat table with no MultiIndex.
display(vix_data)

[*********************100%***********************]  1 of 1 completed


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2012-01-03,^VIX,22.969999,23.100000,22.540001,22.950001,0
1,2012-01-04,^VIX,22.219999,23.730000,22.219999,23.440001,0
2,2012-01-05,^VIX,21.480000,23.090000,21.340000,22.750000,0
3,2012-01-06,^VIX,20.629999,21.719999,20.580000,21.240000,0
4,2012-01-09,^VIX,21.070000,21.780001,21.000000,21.670000,0
...,...,...,...,...,...,...,...
3511,2025-12-18,^VIX,16.870001,17.680000,15.930000,17.610001,0
3512,2025-12-19,^VIX,14.910000,16.530001,14.910000,16.309999,0
3513,2025-12-22,^VIX,14.080000,15.260000,14.030000,15.160000,0
3514,2025-12-23,^VIX,14.000000,14.450000,13.640000,14.090000,0


In [58]:
# 1. Identify dates present in 'data' but NOT in 'vix_data'
missing_vix_dates = data[~data['Date'].isin(vix_data['Date'])]

# 2. Check the results
if not missing_vix_dates.empty:
    print(f"⚠️ Found {len(missing_vix_dates)} days in {ticker} that are missing from VIX data:")
    # Display the specific dates and the ticker values for those days
    display(missing_vix_dates[['Date', 'Ticker', 'Close']])
else:
    print(f"✅ All {len(data)} trading days in {ticker} are present in the VIX dataset.")

# 3. Optional: Detailed check for the first few missing dates (if any)
# This helps identify if it's a specific period or random holidays
if not missing_vix_dates.empty:
    print("\nFirst 5 missing dates:")
    print(missing_vix_dates['Date'].head().tolist())

✅ All 3516 trading days in UVXY are present in the VIX dataset.


In [59]:
def calculate_sma(df, lookback):
    """
    Calculates a simple moving average for a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with a 'close' column.
        lookback (int): The number of periods for the moving average.

    Returns:
        pd.DataFrame: The DataFrame with a new column for the moving average.
    """
    # Create a copy to avoid modifying the original DataFrame
    df_sma = df.copy()

    # Calculate the simple moving average
    df_sma['SMA'] = df_sma['Close'].rolling(window=lookback).mean()

    # Calculate the difference between the SMA and the Close price
    df_sma['SMA_sign'] = np.sign(df_sma['Close'] - df_sma['SMA'])

    return df_sma



In [60]:
def analyze_sma_changes(df, vix_df, lookback, vix_threshold):
    """
    This function simulates a short-volatility strategy by identifying instances where the asset price drops below its Simple Moving Average while the VIX remains above a specified threshold.
    It calculates the return for each 'bearish' period—measured from the initial crossover to the subsequent bullish exit—and aggregates these results into yearly performance metrics
    including total return, trade count, and win rate.

    Args:
        df (pd.DataFrame): The input DataFrame containing historical price data with 'Date' and 'Close' columns.
        vix_df (pd.DataFrame): The DataFrame containing historical VIX Index data with 'Date' and 'Close' columns.
        lookback (int): The number of periods used to calculate the Simple Moving Average.
        vix_threshold (float/int): The minimum VIX level required to trigger a bearish entry signal.

    Returns:
        pd.DataFrame: A summary table grouped by Year, including Lookback, VIX_Threshold, Total_Return,
                      Trade_Count, Success_Count, and Win_Rate.
    """

    # 1. Calculate SMA and signs for every day
    df_with_sma = calculate_sma(df=df.copy(), lookback=lookback)

    # 2. Merge VIX data into the main dataframe based on Date
    # We use 'left' join to keep all price dates and pull in the corresponding VIX close
    vix_subset = vix_df[['Date', 'Close']].rename(columns={'Close': 'VIX_Close'})
    df_with_sma = pd.merge(df_with_sma, vix_subset, on='Date', how='left')

    # 3. Identify ALL sign changes (crosses up AND crosses down)
    previous_sign = df_with_sma['SMA_sign'].shift(1)
    all_crosses_mask = (df_with_sma['SMA_sign'] != previous_sign) & (~previous_sign.isna())

    # Create a DataFrame of every signal event
    all_signals = df_with_sma[all_crosses_mask].copy()

    # 4. Calculate Bearish Return: (Entry Price - Exit Price) / Entry Price
    # Because all_signals contains the NEXT signal (the Bullish cross),
    # shift(-1) now correctly points to the "Exit".
    all_signals.loc[:, 'Next_Close_Return'] = (
        (all_signals['Close'] - all_signals['Close'].shift(-1)) / all_signals['Close']
    )

    # 5. FILTER FOR BEARISH ENTRIES + VIX THRESHOLD
    # - SMA_sign == -1 (Price crossed below SMA)
    # - VIX_Close > vix_threshold (Volatility is high enough)
    bearish_trades = all_signals[
        (all_signals['SMA_sign'] == -1) &
        (all_signals['VIX_Close'] > vix_threshold)
    ].copy()

    # A trade is successful if the return is positive
    bearish_trades['Is_Success'] = bearish_trades['Next_Close_Return'] > 0

    # 6. Clean up and Aggregate
    bearish_trades['Date'] = pd.to_datetime(bearish_trades['Date'])
    bearish_trades['Year'] = bearish_trades['Date'].dt.year

    # Update GroupBy to aggregate Sum, Count (Total Trades), and Sum of Successes (Number of Wins)
    yearly_results = bearish_trades.groupby('Year').agg(
        Total_Return=('Next_Close_Return', 'sum'),
        Trade_Count=('Next_Close_Return', 'count'),
        Success_Count=('Is_Success', 'sum')
    ).reset_index()

    # yearly_results = bearish_trades.groupby('Year')['Next_Close_Return'].sum().reset_index()
    yearly_results['Lookback'] = lookback
    yearly_results['VIX_Threshold'] = vix_threshold

    # Optional: Add Win Rate percentage
    yearly_results['Win_Rate'] = yearly_results['Success_Count'] / yearly_results['Trade_Count']

    return yearly_results[['Year', 'Lookback', 'VIX_Threshold', 'Total_Return', 'Trade_Count', 'Success_Count', 'Win_Rate']]

In [61]:
# Initialize an empty list to store the results from each lookback value
results_list = []

# Iterate through each value in the specified range
for lookback_value in range(lookback_range[0], lookback_range[1] + 1):
    # Calculate SMA changes for the current lookback value
    #df_sma_result = analyze_sma_changes(df=data.copy(), lookback=lookback_value)
    df_sma_result = analyze_sma_changes(df=data.copy(), vix_df=vix_data, lookback=lookback_value, vix_threshold=vix_limit)
     # Append the result to the list
    results_list.append(df_sma_result)

# Concatenate all the DataFrames in the list into a single DataFrame
all_sma_results = pd.concat(results_list, ignore_index=True)

# Print the resulting DataFrame
print("DataFrame with Simple Moving Average analysis for different lookback values:")
display(all_sma_results)

DataFrame with Simple Moving Average analysis for different lookback values:


,Year,Lookback,VIX_Threshold,Total_Return,Trade_Count,Success_Count,Win_Rate
0,2012,3,17,1.513040,26,13,0.500000
1,2013,3,17,0.604204,2,2,1.000000
2,2014,3,17,0.512642,3,3,1.000000
3,2015,3,17,0.918879,13,9,0.692308
4,2016,3,17,0.082158,12,6,0.500000
...,...,...,...,...,...,...,...
219,2021,21,17,0.251577,17,8,0.470588
220,2022,21,17,-0.193160,14,4,0.285714
221,2023,21,17,0.669764,8,4,0.500000
222,2024,21,17,0.061489,6,2,0.333333


In [62]:
# 1. Pivot the DataFrame using 'Trade_Count' as the values
pivoted_trade_results = all_sma_results.pivot(
    index='Year',
    columns='Lookback',
    values='Trade_Count'
)

# 2. Fill missing values with 0 AND convert to integer
# .astype(int) ensures the counts are displayed as whole numbers (e.g., 5 instead of 5.0)
pivoted_trade_results = pivoted_trade_results.fillna(0).astype(int)

# 3. Display the results
print("Pivoted DataFrame: Trade Count by Year and Lookback:")

display(pivoted_trade_results)

Pivoted DataFrame: Trade Count by Year and Lookback:


Lookback,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,
2012,26,24,18,15,15,11,10,9,9,11,13,12,10,10,10,12,11,13,13
2013,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2014,3,3,3,3,3,3,3,2,1,1,1,0,0,0,0,0,0,0,0
2015,13,11,10,11,8,9,8,9,8,8,7,7,6,7,7,7,7,7,6
2016,12,11,10,9,7,6,6,6,7,8,7,7,6,5,5,5,6,6,7
2018,18,14,12,13,12,8,6,8,7,7,8,9,8,8,8,6,6,6,6
2019,12,9,8,10,8,8,10,10,9,7,4,4,5,4,4,4,3,3,3
2020,36,32,28,25,21,20,14,14,13,14,14,14,15,15,13,13,12,12,11
2021,36,33,29,25,21,21,21,22,23,23,23,21,20,22,21,21,20,18,17


In [68]:
# 1. Pivot the DataFrame using 'success_rate' as the values
pivoted_success_results = all_sma_results.pivot(
    index='Year',
    columns='Lookback',
    values='Win_Rate'
)

# 2. Fill missing values (years with no trades) with 0
pivoted_success_results = pivoted_success_results.fillna(0)

# 3. Calculate the average win rate for each lookback and append as a summary row
pivoted_success_results.loc['Average'] = pivoted_success_results.mean()

# 4. Display the results
print("Pivoted DataFrame: Success Rate (Wins / Total Trades) by Year and Lookback:")
# The format string "{:.2%}" handles the display of the new Average row automatically
display(pivoted_success_results.style.format("{:.2%}"))

Pivoted DataFrame: Success Rate (Wins / Total Trades) by Year and Lookback:


Lookback,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,
2012,50.00%,54.17%,72.22%,80.00%,66.67%,72.73%,70.00%,66.67%,66.67%,54.55%,46.15%,50.00%,60.00%,60.00%,60.00%,50.00%,54.55%,46.15%,46.15%
2013,100.00%,100.00%,100.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2014,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,66.67%,100.00%,100.00%,100.00%,100.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
2015,69.23%,45.45%,40.00%,36.36%,50.00%,44.44%,50.00%,44.44%,50.00%,37.50%,42.86%,42.86%,33.33%,28.57%,28.57%,28.57%,28.57%,28.57%,33.33%
2016,50.00%,54.55%,50.00%,55.56%,71.43%,50.00%,50.00%,50.00%,42.86%,37.50%,42.86%,42.86%,50.00%,60.00%,60.00%,60.00%,50.00%,50.00%,42.86%
2018,33.33%,35.71%,41.67%,30.77%,50.00%,62.50%,50.00%,50.00%,57.14%,57.14%,25.00%,22.22%,25.00%,25.00%,25.00%,33.33%,50.00%,50.00%,50.00%
2019,41.67%,44.44%,50.00%,50.00%,37.50%,37.50%,40.00%,40.00%,44.44%,57.14%,75.00%,75.00%,60.00%,50.00%,50.00%,50.00%,66.67%,66.67%,66.67%
2020,52.78%,53.12%,50.00%,56.00%,52.38%,45.00%,50.00%,50.00%,53.85%,50.00%,42.86%,50.00%,53.33%,53.33%,53.85%,53.85%,50.00%,41.67%,45.45%
2021,61.11%,51.52%,41.38%,40.00%,42.86%,42.86%,42.86%,40.91%,39.13%,39.13%,39.13%,42.86%,40.00%,36.36%,42.86%,42.86%,45.00%,50.00%,47.06%


In [64]:
# Pivot the DataFrame
pivoted_sma_results = all_sma_results.pivot(index='Year', columns='Lookback', values='Total_Return')

pivoted_sma_results = pivoted_sma_results.fillna(0)

# Display the pivoted DataFrame
print("Pivoted DataFrame with Lookback as columns, Year as rows, and Total_Return as values:")
display(pivoted_sma_results)

Pivoted DataFrame with Lookback as columns, Year as rows, and Total_Return as values:


Lookback,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,
2012,1.513040,1.954498,2.512444,2.527987,2.526139,2.184868,2.072448,2.147636,2.147636,1.880975,1.666537,1.708198,1.647438,1.647438,1.767707,1.751615,1.765977,1.433402,1.278901
2013,0.604204,0.390902,0.363498,0.553889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2014,0.512642,0.814051,0.691579,0.780664,0.683747,0.685434,0.573388,0.585053,0.368850,0.380939,0.380939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015,0.918879,0.454200,0.162999,0.151332,0.587332,0.380991,0.567812,0.375037,0.402551,0.266848,0.289204,0.464456,0.348038,0.263590,0.229347,0.064347,-0.025538,-0.025538,0.600431
2016,0.082158,0.509737,0.592809,0.804171,1.102695,1.153251,1.069644,1.083293,0.895151,0.627526,0.709298,0.708645,0.965633,1.045826,1.045826,0.929332,1.022373,1.407473,1.397431
2018,-0.656346,-0.118320,0.064336,-0.284781,0.187358,0.241314,0.133371,-0.054594,-0.065126,-0.169368,-0.069450,-0.267654,-0.266822,-0.164944,-0.186085,0.004099,-0.020502,0.039680,0.054229
2019,0.232255,0.314879,0.313365,0.499470,0.349156,0.398430,0.405679,0.354964,0.611681,0.652639,0.808793,0.808793,0.747169,0.375744,0.341741,0.341741,0.403365,0.403365,0.366713
2020,0.667361,0.946321,0.944927,0.638175,0.676449,0.257814,0.660689,0.614415,0.711001,0.416517,0.264488,0.248707,0.106742,0.106742,0.389221,0.363408,0.439383,0.337248,0.419053
2021,0.870967,0.497068,0.042677,0.203824,0.659166,0.226943,0.096090,0.044754,0.019166,0.101211,0.141720,0.209442,-0.124691,-0.048930,0.041387,0.041387,0.094683,0.233350,0.251577


In [65]:
# Ensure 'Date' column in the initial 'data' DataFrame is in datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Extract the year from the 'Date' column
data['Year'] = data['Date'].dt.year

# Group by year and get the first and last close prices
yearly_price_change = data.groupby('Year')['Close'].agg(['first', 'last'])

# Calculate the difference between the last and first close price for each year
yearly_price_change['Yearly_Return'] = (yearly_price_change['first'] - yearly_price_change['last']) / yearly_price_change['first']

# Drop the 'first' and 'last' columns
yearly_price_change = yearly_price_change.drop(columns=['first', 'last'])

# Display the result
print("Difference between the last and first close price of each year:")
display(yearly_price_change)

Difference between the last and first close price of each year:


,Yearly_Return
Year,
2012,0.967984
2013,0.895582
2014,0.639478
2015,0.766954
2016,0.945244
2017,0.931933
2018,-0.729735
2019,0.834574
2020,0.122012


In [66]:
# Join the yearly_price_change DataFrame with the yearly_next_close_diff_sum Series on the 'Year' index
comparison_df = yearly_price_change.join(pivoted_sma_results)

# Display the combined DataFrame, excluding the 'first' and 'last' columns
display(comparison_df)

,Yearly_Return,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,,
2012,0.967984,1.513040,1.954498,2.512444,2.527987,2.526139,2.184868,2.072448,2.147636,2.147636,1.880975,1.666537,1.708198,1.647438,1.647438,1.767707,1.751615,1.765977,1.433402,1.278901
2013,0.895582,0.604204,0.390902,0.363498,0.553889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2014,0.639478,0.512642,0.814051,0.691579,0.780664,0.683747,0.685434,0.573388,0.585053,0.368850,0.380939,0.380939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015,0.766954,0.918879,0.454200,0.162999,0.151332,0.587332,0.380991,0.567812,0.375037,0.402551,0.266848,0.289204,0.464456,0.348038,0.263590,0.229347,0.064347,-0.025538,-0.025538,0.600431
2016,0.945244,0.082158,0.509737,0.592809,0.804171,1.102695,1.153251,1.069644,1.083293,0.895151,0.627526,0.709298,0.708645,0.965633,1.045826,1.045826,0.929332,1.022373,1.407473,1.397431
2017,0.931933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,-0.729735,-0.656346,-0.118320,0.064336,-0.284781,0.187358,0.241314,0.133371,-0.054594,-0.065126,-0.169368,-0.069450,-0.267654,-0.266822,-0.164944,-0.186085,0.004099,-0.020502,0.039680,0.054229
2019,0.834574,0.232255,0.314879,0.313365,0.499470,0.349156,0.398430,0.405679,0.354964,0.611681,0.652639,0.808793,0.808793,0.747169,0.375744,0.341741,0.341741,0.403365,0.403365,0.366713
2020,0.122012,0.667361,0.946321,0.944927,0.638175,0.676449,0.257814,0.660689,0.614415,0.711001,0.416517,0.264488,0.248707,0.106742,0.106742,0.389221,0.363408,0.439383,0.337248,0.419053


In [67]:
# Calculate the sum and standard deviation of each column in the comparison_df
grand_totals = comparison_df.sum()
standard_deviations = comparison_df.std()

# Calculate the Mean Absolute Deviation for each column
mean_absolute_deviations = comparison_df.apply(lambda x: (x - x.mean()).abs().mean())

# Combine the metrics into a single DataFrame for display
summary_df = pd.DataFrame({
    'Grand Total': grand_totals,
    'Standard Deviation': standard_deviations,
    'Mean Absolute Deviation': mean_absolute_deviations
})

# Display the summary DataFrame
print("Grand Totals, Standard Deviations, and Mean Absolute Deviations for each column:")
display(summary_df)

Grand Totals, Standard Deviations, and Mean Absolute Deviations for each column:


,Grand Total,Standard Deviation,Mean Absolute Deviation
Yearly_Return,8.729193,0.458569,0.310603
3,5.856166,0.552378,0.391161
4,6.846332,0.533398,0.348613
5,7.496995,0.642640,0.400756
6,7.961337,0.656860,0.372189
7,8.711849,0.630820,0.355148
8,7.454119,0.568189,0.367589
9,6.958762,0.554544,0.372578
10,5.808965,0.617069,0.424479
11,5.825138,0.605342,0.421109
